In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

CCAL version 0.9.0 @ /home/kwatme/github/ccal/ccal/__init__.py


In [3]:
if SETTING["signal_type"] == "context":

    raise

RuntimeError: No active exception to reraise

In [ ]:
feature_x_sample_processed = pd.read_table(
    PATH["feature_x_sample.processed.tsv"], index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
title_template = "{} Ranking by Raw Signal"

value_name = "Raw Signal Standard Deviation"

In [ ]:
if SETTING["select_feature_automatically"]:

    features = ccal.select_series_indices(
        feature_x_sample_processed.std(axis=1),
        ">",
        standard_deviation=1,
        title=dict(text=title_template.format(feature_x_sample_processed.index.name)),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name),
    )

else:

    features = feature_x_sample_processed.index

features.size

In [ ]:
if SETTING["select_sample_automatically"]:

    samples = ccal.select_series_indices(
        feature_x_sample_processed.std(),
        ">",
        standard_deviation=1,
        title=dict(text=title_template.format(feature_x_sample_processed.index.name)),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name),
    )

else:

    samples = feature_x_sample_processed.columns

samples.size

In [ ]:
raw_signal = feature_x_sample_processed.loc[features, samples]

In [ ]:
if SETTING["raw_signal_normalization_method"] is not None:

    raw_signal = ccal.normalize_df(
        raw_signal,
        SETTING["raw_signal_normalization_axis"],
        SETTING["raw_signal_normalization_method"],
        raise_for_bad=False,
    )

In [ ]:
raw_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

ccal.summarize_feature_x_sample(
    raw_signal,
    feature_x_sample_alias=SETTING["feature_x_sample_alias"],
    feature_x_sample_value_name="Signal",
    plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
    plot_distributions_max_size=SETTING["plot_distributions_max_size"],
    plot_rug_max_size=SETTING["plot_rug_max_size"],
)